In [ ]:
import os
import Cell_BLAST as cb
import utils
os.environ["CUDA_VISIBLE_DEVICES"] = utils.pick_gpu_lowest_memory()
cb.config.RANDOM_SEED = 0
cb.config.N_JOBS = 4
fixed_model_kwargs = dict(
    latent_dim=10, cat_dim=20,
    epoch=500, patience=20
)

In [ ]:
cb.__version__

---
# Mouse
## Quake_Smart-seq2_Fat

In [ ]:
quake_smart_seq2_fat = cb.data.ExprDataSet.read_dataset("../../Datasets/data/Quake_Smart-seq2_Fat/data.h5")
utils.peek(quake_smart_seq2_fat, "build/fat/Quake_Smart-seq2_Fat")
quake_smart_seq2_fat.obs.head()

In [ ]:
quake_smart_seq2_fat.obs["cluster"] = "cluster_" + quake_smart_seq2_fat.obs["cluster"].astype(int).astype(str)
quake_smart_seq2_fat.obs.dtypes

In [ ]:
opt_model_kwargs = dict(batch_effect="donor")
quake_smart_seq2_fat_model = cb.directi.fit_DIRECTi(
    quake_smart_seq2_fat, quake_smart_seq2_fat.uns["seurat_genes"],
    **fixed_model_kwargs, **opt_model_kwargs
)
quake_smart_seq2_fat.latent = quake_smart_seq2_fat_model.inference(quake_smart_seq2_fat)

In [ ]:
ax = quake_smart_seq2_fat.visualize_latent("cell_ontology_class", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/fat/Quake_Smart-seq2_Fat/cell_ontology_class.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = quake_smart_seq2_fat.visualize_latent("cell_type1", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/fat/Quake_Smart-seq2_Fat/cell_type1.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = quake_smart_seq2_fat.visualize_latent("cluster", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/fat/Quake_Smart-seq2_Fat/cluster.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = quake_smart_seq2_fat.visualize_latent("free_annotation", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/fat/Quake_Smart-seq2_Fat/free_annotation.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = quake_smart_seq2_fat.visualize_latent("region", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/fat/Quake_Smart-seq2_Fat/region.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = quake_smart_seq2_fat.visualize_latent("donor", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/fat/Quake_Smart-seq2_Fat/donor.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = quake_smart_seq2_fat.visualize_latent("gender", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/fat/Quake_Smart-seq2_Fat/gender.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
quake_smart_seq2_fat.write_dataset("build/fat/Quake_Smart-seq2_Fat/Quake_Smart-seq2_Fat.h5")

In [ ]:
%%capture capio
quake_smart_seq2_fat_models = [quake_smart_seq2_fat_model]
opt_model_kwargs = dict(batch_effect="donor")
for i in range(1, cb.config.N_JOBS):
    print("==== Model %d ====" % i)
    quake_smart_seq2_fat_models.append(cb.directi.fit_DIRECTi(
        quake_smart_seq2_fat, quake_smart_seq2_fat.uns["seurat_genes"],
        **fixed_model_kwargs, **opt_model_kwargs,
        random_seed=i
    ))
quake_smart_seq2_fat_blast = cb.blast.BLAST(quake_smart_seq2_fat_models, quake_smart_seq2_fat)
quake_smart_seq2_fat_blast.save("build/fat/Quake_Smart-seq2_Fat")

In [ ]:
with open("build/fat/Quake_Smart-seq2_Fat/stdout.txt", "w") as f:
    f.write(capio.stdout)
with open("build/fat/Quake_Smart-seq2_Fat/stderr.txt", "w") as f:
    f.write(capio.stderr)

In [ ]:
utils.self_projection(quake_smart_seq2_fat_blast, "build/fat/Quake_Smart-seq2_Fat")

In [ ]:
%%writefile build/fat/Quake_Smart-seq2_Fat/predictable.txt
cell_ontology_class
cell_type1
cluster
free_annotation
region